In [26]:
import pandas as pd
import numpy as np
import ast
from tqdm import tqdm
import sys
import time

from flashgeotext.geotext import GeoText
from geopy.geocoders import Nominatim

In [3]:
articles = pd.read_excel('data/comparative_labelled.xlsx', index_col=0)

In [4]:
articles.authors = articles.authors.apply(lambda x: ast.literal_eval(x) if type(x) == str else x)
articles.author_affils = articles.author_affils.apply(lambda x: ast.literal_eval(x) if type(x) == str else x)
articles.keywords = articles.keywords.apply(lambda x: ast.literal_eval(x) if type(x) == str else x)
articles.references_pmids = articles.references_pmids.apply(lambda x: ast.literal_eval(x) if type(x) == str else x)
articles.mesh_terms = articles.mesh_terms.apply(lambda x: ast.literal_eval(x) if type(x) == str else x)

In [5]:
articles['affil_countries'] = np.nan
articles['affil_countries'] = articles['affil_countries'].astype(object)
articles['affil_countries_unique'] = np.nan
articles['affil_countries_unique'] = articles['affil_countries'].astype(object)
articles['affil_first_country'] = np.nan
articles['affil_last_country'] = np.nan

In [ ]:
def find_affil_countries(affils: list, retry_count = 5):
    
    geolocator = Nominatim(user_agent='health_ai_scraper')
    geotext = GeoText()
    
    if affils == affils: # Check to make sure not NaN
        country_list = []
        location = None
        
        last_affil = None
        last_country = None
        
        try_count = 0
    
        for affil in affils:
            if affil == last_affil: # Check to see if we've seen this before and take a shortcut if we have
                country_list = country_list + last_country

            else:
                while try_count < retry_count:
                    try:
                        last_affil = affil # Set that we've examined this affil
                
                        countries = [*geotext.extract(input_text=affil, span_info=False)['countries'].keys()] # Look for countries
                
                        if (len(countries) == 0): # If we dont find a country look harder
                            cities = [*geotext.extract(input_text=affil, span_info=False)['cities'].keys()]
                            if len(cities) > 0:
                                location = geolocator.geocode(cities[-1])
                            else:
                                location = geolocator.geocode(' '.join(affil.split(" ")[-2:]))
                                if location == None:
                                    location = geolocator.geocode(affil)
                                    if location == None:
                                        tqdm.write("Can't find a country for:")
                                        tqdm.write(affil)
                                        country_list = country_list + [np.nan]
                                        last_country = [np.nan]
                            
                        else: # If we do find a country then att it to the list and set the last_country variable
                            country_list = country_list + countries
                            last_country = countries
        
                        if location != None: # If we found an address using the other search techniques
                            countries = [*geotext.extract(input_text=location.address, span_info=False)['countries'].keys()]
                            country_list = country_list + countries
                            last_country = countries
                            
                        break
                            
                    except:
                        try_count += 1
                        tqdm.write(f"Error parsing {affil}, trying again for a maximum of 5 times.")
                        
    else: # If the affil is NaN then make the country list NaN
        country_list = [np.nan]
        
    unique_countries = list(set(country_list))
    first_affil_country = country_list[0]
    last_affil_country = country_list[-1]    
        
    return country_list, unique_countries, first_affil_country, last_affil_country

In [ ]:
def parse_affil_countries(df, max_consecutive_failures = 5, filter_column = 'include'):
    
    consecutive_failures = 0
    
    country_df = df.copy()
    
    country_df['affil_countries'] = np.nan
    country_df['affil_countries_unique'] = np.nan
    country_df['affil_countries_first'] = np.nan
    country_df['affil_countries_last'] = np.nan
    
    with tqdm(total=country_df.shape[0], file=sys.stdout) as pbar:
        for row in country_df[country_df[filter_column] == 1].itertuples():

            try:
                affils = row.author_affils
    
                country_list, unique_countries, first_affil_country, last_affil_country = find_affil_countries(affils)
    
                country_df.loc[row.Index, 'affil_countries'] = str(country_list)
                country_df.loc[row.Index, 'affil_countries_unique'] = str(list(set(country_list)))
                country_df.loc[row.Index, 'affil_countries_first'] = country_list[0]
                country_df.loc[row.Index, 'affil_countries_last'] = country_list[-1]
            
                consecutive_failures = 0
        
            except Exception as e:
                tqdm.write(e)
                consecutive_failures += 1
                if consecutive_failures >=  max_consecutive_failures:
                    tqdm.write("Failed too many in a row, something is broken, stopping and returning possibly partially labelled DF...")
                    break
                
            pbar.update(1)
            
    country_df.replace("[nan]", np.nan, inplace=True)
            
    return country_df

In [30]:
consecutive_failures = 0

with tqdm(total=articles.shape[0], file=sys.stdout) as pbar:
    for row in articles.itertuples():

        try:
            affils = row.author_affils
    
            country_list, unique_countries, first_affil_country, last_affil_country = find_affil_countries(affils)
    
            articles.loc[row.Index, 'affil_countries'] = str(country_list)
            articles.loc[row.Index, 'affil_countries_unique'] = str(list(set(country_list)))
            articles.loc[row.Index, 'affil_first_country'] = country_list[0]
            articles.loc[row.Index, 'affil_last_country'] = country_list[-1]
            
            consecutive_failures = 0
        
        except:
            consecutive_failures += 1
            if consecutive_failures > 5:
                print("Failed too many in a row, something is broken, stopping...")
                break
            time.sleep(5)
                
        pbar.update(1)

Can't find a country for:                            
Visulytix Ltd.                                       
Can't find a country for:                             
Internal Medicine-Hematology/Oncology, and.          
Can't find a country for:                             
Institute for Datability Science.                    
Can't find a country for:                             
Radiology Department, Hospital Universitarioy Polite'cnico La Fe and Biomedical Imaging Research Group (GIBI230).
Can't find a country for:                             
Lunit Incorporated.                                  
Can't find a country for:                             
Mater Dei Hospital, Triq Dun Karm, L'Imsida, Malta.   
Can't find a country for:                             
Department of Radiology (S. Mutasa, S. Marfatia, A.L.).
Can't find a country for:                            
Department of Neurological Surgery (J.L.S., E.S.C.). 
Can't find a country for:                            
Department of